# Decorators


A decorator involves concepts of functional programming. Although it is the same underlying concept, **we have two different types of decorators** in Python:

* **functional** decorators.

* **class** decorators

A **decorator** in Python is **any** callable Python object that is used **to modify a function or a class**. A reference to a `func` function or a `C` class is passed to a decorator and the decorator returns a modified function or class. Modified functions or classes generally contain calls to the original `func` function or `C` class.

First, **function names are references** to functions and **we can assign multiple names to the same function**. It is possible to **nest functions** within functions. It is also **possible** to make one **function** the **argument of another**.

The **decoration occurs on the line before the function header**. The `@` is **followed** by the **decorator function name**.

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# show all ouputs of the same cell in the notebook, not only the last

In [4]:
def our_decorator(func):
    def function_wrapper(x):
        print("Before calling {}".format(func.__name__))
        func(x)
        print("After calling {}".format(func.__name__))
    return function_wrapper

@our_decorator
def foo(x):
    print("foo has been called with: {}".format(x))
    
foo('he hey')

Before calling foo
foo has been called with: he hey
After calling foo


It can also be used with functions already created in any package.

In [9]:
from math import sin, cos

def our_decorator(func):
    def function_wrapper(x):
        print("Before calling {}".format(func.__name__))
        res = func(x)
        print(res)
        print("After calling {}".format(func.__name__))
    return function_wrapper

# Alternative way to invoke the decorator
sin = our_decorator(sin)
cos = our_decorator(cos)

for f in [sin, cos]:
    f(3.1415)

Before calling sin
9.265358966049026e-05
After calling sin
Before calling cos
-0.9999999957076562
After calling cos


The `function_wrapper` above only works for **functions with exactly one parameter**. **We provide a generalized version** of `function_wrapper`, **which accepts functions with arbitrary parameters** in the following example.

In [10]:
from random import randint, choice

def our_decorator(func):
    def function_wrapper(*args, **kwargs):
        print("Before calling {}".format(func.__name__))
        res = func(*args, **kwargs)
        print(res)
        print("After calling {}".format(func.__name__))
    return function_wrapper

# Decorated functions
randint = our_decorator(randint)
choice = our_decorator(choice)

randint(3, 8)
choice([4, 5, 6])

Before calling randint
8
After calling randint
Before calling choice
6
After calling choice


To summarize, we can say that a decorator in Python is a callable Python object that is used to modify a function, method or class definition. The original object, the one to be modified, is passed to a decorator as an argument. The decorator returns a modified object, e.g. a modified function, which is bound to the name used in the definition.

Let's see some interesting applications. For argument parsing, check that the input number for calculating the factorial is natural.

In [4]:
def argument_test_natural_number(f):
    def helper(x):
        if type(x) == int and x > 0:
            return f(x)
        else:
            raise Exception("Argument is not an integer")
    return helper
    
@argument_test_natural_number
def factorial(n):
    if n == 1:
        return 1
    else:
        return n * factorial(n-1)
        # recurrence function

factorial(4)
factorial(-1)

24

Exception: Argument is not an integer

To count the number of times a function has been invoked.

In [10]:
def call_counter(func):
    def helper(*args, **kwargs):
        helper.calls += 1
        # method to count the calls
        return func(*args, **kwargs)
    helper.calls = 0
    return helper

@call_counter
def succ(x):
    return x + 1

@call_counter
def mul1(x, y=1):
    return x*y + 1


for i in range(3):
    mul1(i,i+1)
print(f'Number of calls {mul1.calls}')

1

3

7

Number of calls 3


It is possible to make the **decorator depend on a parameter** that **particularizes the decoration according to the need**. We will achieve this by **wrapping the decorator in a function**, which could already depend on several parameters although in the example we make it depend on only one.

In [12]:
def greeting(expr):
    # function that wraps the decorator
    def greeting_decorator(func):
        # decorator function, note that it has func argument
        def function_wrapper(x):
            # real action performed
            print(f'I say {expr}, a my function {func.__name__} returns: ')
            func(x)
        return function_wrapper
    return greeting_decorator

@greeting("καλημερα")
def foo(x):
    print('Another thing')

foo('Needs an useless argument')

I say καλημερα, a my function foo returns: 
Another thing


In [14]:
# Alternative way
def foo(x):
    print('Another thing')

greeting2 = greeting("καλημερα")
foo = greeting2(foo)
foo('Needs an useless argument')

I say καλημερα, a my function foo returns: 
Another thing


By using this previous dependency on a function, a decorator can show the results of a generator.

In [15]:
def go_throughInf(n):
    def go_through(gen):
        # note that gen is the object to wrap
        def gtAux(*args, **kwargs):
            g = gen(*args, **kwargs)
            # expect random number of inputs
            sentinel = True
            for _ in range(n):
                try:
                    print(next(g))
                except StopIteration:
                    print('empty generator')
        return gtAux
    return go_through

@go_throughInf(10)
# display first 10 number of fibonacci
def fib(x=0,y=1):
    a, b = x, y
    while True:
        yield a
        a, b = b, a + b
        
fib()

0
1
1
2
3
5
8
13
21
34


In [17]:
1e6

1000000.0

To measure the performance of a function in terms of time and resources.

In [31]:
import psutil
import time
import os

def manytimeit(num):
    def timeit(func):
        def timed(*args, **kw):
            accumulatortime = []
            accumulatorcpu = []
            for _ in range(num):
                ts = time.time()
                pid = os.getppid()
                result = func(*args, **kw)
                te = time.time()
                p = psutil.Process(pid)
                nowcpu = p.cpu_percent(interval=1) / psutil.cpu_count()
                accumlatortime = accumulatortime.append(te-ts)
                accumlatorcpu = accumulatorcpu.append(nowcpu)
            meantime = sum(accumulatortime)/len(accumulatortime)
            meancpu = sum(accumulatorcpu)/len(accumulatorcpu)
    #         maxmem = psutil.virtual_memory()[3]/(1e6)
            # total memory in use for the system
#             info = (pid, func.__name__, meantime, maxcpu)
            print(f'PID:{pid}\nName:{func.__name__}\nTime:{meantime:.10f} sec over {num} times\nCPU:{meancpu:.3f} %')
            return result
        return timed
    return timeit

@manytimeit(3)
# perform that number of times the timing
def fibIte(n):
    def fibonacciIte(n):
        a, b = 0, 1
        for i in range(n):
            a, b = b, a + b
        return a
    return fibonacciIte(n)

fibIte(10)


PID:2692
Name:fibIte
Time:0.0099952221 sec over 1 times
CPU:1.925 %


55

To study in detail the memory consumed by each line of your code you must run it as a script and use the `memory_profiler` package. Run `python example.py` when by using the below decorator, and when not explicitly adding the package just use `python -m memory_profiler example.py`.

In [3]:
from memory_profiler import profile

@profile
def fibIte(n):
    def fibonacciIte(n):
        a, b = 0, 1
        for i in range(n):
            a, b = b, a + b
        return a
    return fibonacciIte(n)


When you want to access certain attributes of the function being wrapped the best solution is the following.

In [33]:
from functools import wraps
# this function wraps the attributes of the main function to avoid presenting the attributes of the decorator

def greeting(func):
    @wraps(func)
    def function_wrapper(x):
        """ function_wrapper of greeting """
        print("Hi, " + func.__name__ + " returns:")
        return func(x)
    return function_wrapper

@greeting
def f(x):
    """ just some silly function """
    return x + 4

print(f(10))
print("function name: " + f.__name__)
print("docstring: " + f.__doc__)
print("module name: " + f.__module__)

Hi, f returns:
14
function name: f
docstring:  just some silly function 
module name: __main__


So far we have used functions as decorators. Before we can define a **decorator as a class**, **we have to introduce the class method** `__call__`. We already mentioned that a decorator is simply an invocable object that takes a function as an input parameter. **A function is an invocable object, but many Python programmers do not know that there are other invocable objects**. An invocable object is an object that can be used and behaves like a function but might not be a function. It is possible to **define classes** so that **instances** are **invocable objects**. The `__call__` method is called, if the instance is called "like a function", i.e., using parentheses.

In [34]:
class decorator2:
    
    def __init__(self, f):
        self.f = f
        print('Initializing decorator class')
        
    def __call__(self):
        print("Decorating", self.f.__name__)
        self.f()

@decorator2
def foo():
    print("inside foo()")

foo()

Initializing decorator class
Decorating foo
inside foo()
